# Notebook 1 : Prise en main de scikit-learn

Notebook préparé par [Chloé-Agathe Azencott](http://cazencott.info).

Ce notebook vous permettra de découvrir des fonctionalités de `scikit-learn` permettant :
* d'entrainer et évaluer un algorithme d'apprentissage supervisé
* d'encoder des variables qualitatives ;
* de ramener des variables à une fourchette de valeurs ;
* de transformer des variables pour rapprocher leur distribution de celle d'une gaussienne.

In [ ]:
# charger numpy as np, matplotlib as plt
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.rc('font', **{'size': 12}) # règle la taille de police globalement pour les plots (en pt)

In [ ]:
import pandas as pd

## 1. Chargement des données
Dans ce notebook nous allons travailler avec les données contenues dans le fichier `data/auto-mpg.tsv`. Ces données, obtenues sur https://archive.ics.uci.edu/ml/datasets/Auto+MPG, décrivent des voitures par les variables suivantes :

    1. mpg:           consommation (en miles par gallon), continue
    2. cylinders:     nombre de cylindres, discrète
    3. displacement:  cylindrée, continue
    4. horsepower:    chevaux-vapeur, continue
    5. weight:        poids, continue
    6. acceleration:  accélération, continue
    7. model year:    année, discrète
    8. origin:        région d'origine, discrète (1=Amérique du Nord ; 2=Europe ; 3=Asie)
    9. car name:      nom, chaîne de caractères.

Notre but va être de prédire la consommation d'un véhicule à partir des autres variables (à l'exclusion du nom de la voiture, qui est un identifiant unique).

In [ ]:
# Charger les données
df = pd.read_csv("data/auto-mpg.tsv", delimiter='\t')

__Alternativement :__ Si vous avez besoin de télécharger le fichier (par exemple sur colab) :

In [ ]:
df.head(10)

### Création des matrices X et y de données

In [ ]:
X = np.array(df.drop(columns=['mpg', 'car name']))

In [ ]:
y = np.array(df['mpg'])

In [ ]:
X.shape

In [ ]:
y.shape

## 2. Visualisation des données

Nous allons maintenant visualiser les variables représentant nos véhicules. Pour ce faire, nous allons séparer les variables continues (que nous allons représenter chacune par un histogramme) des variables discrètes (que nous allons représenter chacune par un diagramme en barre).

N'hésitez pas à ajuster les paramètres des méthodes de `matplotlib` pour produire des graphiques plus lisibles.

In [ ]:
continuous_features = ['displacement', 'horsepower', 'weight', 'acceleration']
discrete_features = ['cylinders', 'model year', 'origin']

features = list(df.drop(columns=['mpg', 'car name']).columns)

continuous_features_idx = [features.index(feat_name) for feat_name in continuous_features]
discrete_features_idx = [features.index(feat_name) for feat_name in discrete_features]

### Histogrammes pour les variables continues

In [ ]:
fig = plt.figure(figsize=(8, 6))

for (plot_idx, feat_idx) in enumerate(continuous_features_idx):
    # créer un graphique à la position (plot_idx+1) d'une grille 2x2
    ax = fig.add_subplot(2, 2, (plot_idx+1))
    # afficher l'histogramme de la variable feat_idx
    h = ax.hist(X[:, feat_idx], bins=30, edgecolor='none')
    # utiliser le nom de la variable comme titre
    ax.set_title(features[feat_idx])
# espacement entre les graphiques
fig.tight_layout(pad=1.0)

### Diagrammes en barres pour les variables discrètes

In [ ]:
fig = plt.figure(figsize=(12, 3))

for (plot_idx, feat_idx) in enumerate(discrete_features_idx):
    # créer un graphique à la position (plot_idx+1) d'une grille 1x3
    ax = fig.add_subplot(1, 3, (plot_idx+1))

    feature_values = np.unique(X[:, feat_idx])
    frequencies = [(float(len(np.where(X[:, feat_idx]==value)[0]))/X.shape[0]) \
                   for value in feature_values]
    
    b = ax.bar(range(len(feature_values)), frequencies, width=0.5, 
               tick_label=list([int(n) for n in feature_values]))
    
    # utiliser le nom de la variable comme titre
    ax.set_title(features[feat_idx])
fig.tight_layout(pad=1.0)

__Question :__ Observez les ordres de grandeur / fourchettes de valeur des différentes variables.

### Histogramme des étiquettes

In [ ]:
plt.hist(y, bins=30, edgecolor='none')
plt.title('mpg')

## 3. Régression linéaire

Nous allons maintenant utiliser `scikit-learn` pour entraîner une régression linéaire sur les données.

Les modèles linéaires de `scikit-learn` sont implémentés dans le module [`sklearn.linear_model`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model). __N'hésitez pas à vous référer fréquemment à la documentation de scikit-learn, qui est très complète.__

In [ ]:
from sklearn import linear_model

### Entraînement du modèle

In [ ]:
# Initialisation d'un objet LinearRegression
predictor = linear_model.LinearRegression()

In [ ]:
# Entrainement de cet objet sur les données 
predictor.fit(X, y)

### Prédictions
Nous pouvons maintenant utiliser ce modèle pour _prédire_ des étiquettes à partir des variables. En particulier, on peut l'appliquer aux données que l'on vient d'utiliser pour l'entraînement :

In [ ]:
y_pred = predictor.predict(X)

__ATTENTION__ En pratique, ce qui nous intéresse vraiment est la capacité d'un modèle à faire de bonnes prédictions sur des données qui n'ont _pas_ été utilisées pour l'entraîner. La performance d'un modèle sur les données qui ont servi à l'entraîner ne permet pas de déterminer s'il s'agit d'un bon modèle. Nous en discuterons plus en détails dans la suite du cours.

### Performance

Il s'agit maintenant d'évaluer ce modèle.

Pour cela, nous allons utiliser les fonctionalités du module [https://scikit-learn.org/stable/modules/classes.html?highlight=metrics#module-sklearn.metrics](`metrics`) de `scikit-learn`.

Comme il s'agit d'un problème de régression, nous allons utiliser la __RMSE__ (_Root Mean Squared Error_) comme mesure de la performance du modèle : il s'agit de la racine carrée de la moyenne des carrés des erreurs. On utilise la racine carrée pour des questions d'homogénéité : la RMSE s'exprime dans la même unité que l'étiquette.

In [ ]:
from sklearn import metrics

In [ ]:
print("RMSE: %.2f" % metrics.root_mean_squared_error(y, y_pred))

__Question :__ Que pensez-vous de cette erreur ? Est-elle faible? Grande ?

### Visualisation

Nous pouvons aussi utiliser une visualisation, et représenter chaque individu du jeu de test par son étiquette prédite vs. sa vraie étiquette.

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(y, y_pred)

plt.xlabel("Consommation réelle (mpg)")
plt.ylabel("Consommation prédite (mpg)")
plt.title("Régression linéaire")

# Mêmes valeurs sur les deux axes
axis_min = np.min([np.min(y), np.min(y_pred)])-1
axis_max = np.max([np.max(y), np.max(y_pred)])+1
plt.xlim(axis_min, axis_max)
plt.ylim(axis_min, axis_max)
  
# Diagonale y=x
plt.plot([axis_min, axis_max], [axis_min, axis_max], 'k-')

### Coefficients de régression

Pour comprendre notre modèle, nous pouvons regarder les coefficients affectés à chaque variable dans le modèle linéaire appris.

In [ ]:
# Afficher, pour chaque variable, la valeur absolue de son coefficient dans le modèle
num_features = X.shape[1]
feature_names = df.drop(columns=['mpg', 'car name']).columns
plt.scatter(range(num_features), np.abs(predictor.coef_))

plt.xlabel('Variable')
tmp = plt.xticks(range(num_features), feature_names, rotation=90)
tmp = plt.ylabel('Coefficient')

__Question :__ Quelle variable a le plus fort coefficient (en valeur absolue) ? Pensez-vous que cela signifie que cette variable joue un rôle très important dans la prédiction ?

## 4. Changement d'échelle des variables

Le fait que les variables soient sur des échelles différentes rend l'interprétation des coefficients de la régression linéaire délicate. 

### Transformation des variables

Centrer (ramener à une moyenne de 0) et réduire (ramener à un écart-type de 1) les variables permet de remédier à ce problème.

In [ ]:
from sklearn import preprocessing

In [ ]:
standard_scaler = preprocessing.StandardScaler()
standard_scaler.fit(X)

In [ ]:
X_scaled = standard_scaler.transform(X)

### Visualisation des nouvelles variables

#### Histogrammes pour les variables continues
On remplace ici `X` par `X_scaled` dans le code utilisé précédemment.

In [ ]:
fig = plt.figure(figsize=(8, 6))

for (plot_idx, feat_idx) in enumerate(continuous_features_idx):
    # créer un graphique à la position (plot_idx+1) d'une grille 2x2
    ax = fig.add_subplot(2, 2, (plot_idx+1))
    # afficher l'histogramme de la variable feat_idx
    h = ax.hist(X_scaled[:, feat_idx], bins=30, edgecolor='none')
    # utiliser le nom de la variable comme titre
    ax.set_title(features[feat_idx])
# espacement entre les graphiques
fig.tight_layout(pad=1.0)

#### Diagrammes en barres pour les variables discrètes
On remplace ici `X` par `X_scaled` dans le code utilisé précédemment.

In [ ]:
fig = plt.figure(figsize=(12, 3))

for (plot_idx, feat_idx) in enumerate(discrete_features_idx):
    # créer un graphique à la position (plot_idx+1) d'une grille 1x3
    ax = fig.add_subplot(1, 3, (plot_idx+1))

    feature_values = np.unique(X_scaled[:, feat_idx])
    frequencies = [(float(len(np.where(X_scaled[:, feat_idx]==value)[0]))/X_scaled.shape[0]) \
                   for value in feature_values]
    
    b = ax.bar(range(len(feature_values)), frequencies, width=0.5, 
               tick_label=list(['%.1f' % n for n in feature_values]))
    
    # utiliser le nom de la variable comme titre
    ax.set_title(features[feat_idx])
fig.tight_layout(pad=1.0)

### Impact sur le modèle

Nous pouvons maintenant entraîner un modèle `predictor_scaled` sur les données centrées-réduites.

In [ ]:
# Créer un nouvel objet LinearRegression 
predictor_scaled = linear_model.LinearRegression()

# Entraîner predictor_dummy sur les nouvelles données
predictor_scaled.fit(X_scaled, y)

Et créer un array `y_pred_scaled` qui contient les prédictions de `predictor_scaled` sur les données.

In [ ]:
y_pred_scaled = predictor_scaled.predict(X_scaled)

#### RMSE

La RMSE de ce nouveau modèle est :

In [ ]:
print("RMSE (scaled): %.2f" % metrics.root_mean_squared_error(y, y_pred_scaled))

__Question :__ La comparer à la RMSE précédente. Les prédictions sont-elles différentes ?

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(y_pred_scaled, y_pred)

plt.xlabel("Consommation prédite sur les données centrées-réduites (mpg)")
plt.ylabel("Consommation prédite (mpg)")
plt.title("Régression linéaire")

# Mêmes valeurs sur les deux axes
axis_min = np.min([np.min(y), np.min(y_pred)])-1
axis_max = np.max([np.max(y), np.max(y_pred)])+1
plt.xlim(axis_min, axis_max)
plt.ylim(axis_min, axis_max)
  
# Diagonale y=x
plt.plot([axis_min, axis_max], [axis_min, axis_max], 'k-')

#### Comparaison des coefficients de régression.
Enfin, nous pouvons comparer les coefficients de régression des deux modèles. 

In [ ]:
# Afficher, pour chaque variable, la valeur absolue de son coefficient dans le modèle
num_features = X.shape[1]
plt.scatter(range(num_features), np.abs(predictor.coef_), label='Originales')

plt.scatter(range(num_features), np.abs(predictor_scaled.coef_), label='Centrées-réduites', marker='v')

plt.xlabel('Variable')
tmp = plt.xticks(range(num_features), feature_names, rotation=90)
tmp = plt.ylabel('Coefficient')
plt.legend(loc=(0.02, 0.75))

On peut noter que, même si la RMSE est la même, le fait de centrer-réduire les variables a changé la valeur des paramètres appris par le modèle. On peut comparer par exemple les valeurs prises par l'intercept (terme indépendant dans le modèle linéaire).

In [ ]:
predictor.intercept_

In [ ]:
predictor_scaled.intercept_

__Question :__ Quelles sont maintenant les variables les plus pertinentes pour prédire la consommation d'un véhicule ?

## 5. Encodage des variables qualitatives
La variable `origin` est une variable qualitative : l'encodage 1-2-3 est tout à fait arbitraire. Il suppose en particulier, si on réfléchit en termes de distances, que l'Asie est deux fois plus loin de l'Amérique du Nord que de l'Europe, ce qui n'a aucun sens.

Un encodage plus raisonnable pour ce genre de cas est ce qu'on appelle l'encodage _one-hot_, ou encore _dummy encoding_ : on représente la variable par autant de variables binaires qu'il y a de valeurs possibles (3 dans le cas de la variable `origin` : la première correspond à Amérique du Nord, la deuxième à Europe, la troisième à Asie), et on met à `1` la seule de ces variables binaires correspondant à la valeur que l'on encode.

Ainsi l'unique variable `origin` devient 3 variables binaires:
```    
   Amérique du Nord --> 1, 0, 0
   Europe --> 0, 1, 0
   Asie --> 0, 0, 1
```  
Cette représentation a l'inconvénient d'augmenter le nombre de variables, mais les distances euclidiennes sont maintenant plus raisonnables (elles valent 1 si les valeurs sont différentes et 0 si elles sont identiques).

Cette fonctionalité existe dans `pandas` comme dans `scikit-learn`. 

### Transformation one-hot

In [ ]:
# Créer un nouveau data frame où la colomne 'origin' est remplacée par son encodage 'one-hot'
df_dummies = pd.get_dummies(df, columns=['origin'])

In [ ]:
df_dummies.head()

In [ ]:
# Extraire de nouveau les données
X_dummies = np.array(df_dummies.drop(columns=['mpg', 'car name']))

Comme précédemment, on normalise chacune des variables.

In [ ]:
standard_scaler_dummies = preprocessing.StandardScaler()
standard_scaler_dummies.fit(X_dummies)
X_scaled_dummies = standard_scaler_dummies.transform(X_dummies)

### Impact sur le modèle

Nous allons maintenant apprendre une régression linéaire sur les données où la variable `origin` a été remplacée par son encodage one-hot. 

Pour cela, nous créons une instance `predictor_dummy` de la classe `LinearRegression` entraînée sur les données contenant la version _one-hot_ de la variable `origin`.

In [ ]:
# Créer un nouvel objet LinearRegression 
predictor_dummy = linear_model.LinearRegression()

# Entraîner predictor_dummy sur les nouvelles données
predictor_dummy.fit(X_scaled_dummies, y)

Nous pouvons maintenant créer un array `y_pred_dummy` qui contient les prédictions de la nouvelle régression linéaire sur les données.

In [ ]:
y_pred_dummy = predictor_dummy.predict(X_scaled_dummies)

La RMSE de ce nouveau modèle est :

In [ ]:
print("RMSE (encodage one-hot): %.2f" % metrics.root_mean_squared_error(y, y_pred_dummy))

__Question :__ La comparer à la RMSE précédente.

#### Comparaison aux prédictions précédentes

Les performances sont-elles vraiment différentes ? Nous pouvons comparer les prédictions directement.

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(y_pred, y_pred_dummy)

plt.xlabel("Consommation prédite (mpg) (baseline)")
plt.ylabel("Consommation prédite (mpg) (avec one-hot)")
plt.title("Régression linéaire")

# Mêmes valeurs sur les deux axes
axis_min = np.min([np.min(y_pred), np.min(y_pred_dummy)])-1
axis_max = np.max([np.max(y_pred), np.max(y_pred_dummy)])+1
plt.xlim(axis_min, axis_max)
plt.ylim(axis_min, axis_max)
  
# Diagonale y=x
plt.plot([axis_min, axis_max], [axis_min, axis_max], 'k-')

In [ ]:
import scipy.stats as st

In [ ]:
r, pval = st.pearsonr(y_pred, y_pred_dummy)
print("Corrélation entre les prédictions : %.2f (p=%.2e)" % (r, pval))

#### Comparaison des coefficients de régression

Comparons maintenant les deux modèles visuellement :

In [ ]:
# Afficher, pour chaque variable, la valeur absolue de son coefficient dans le modèle
num_features = X.shape[1]
plt.scatter(range(num_features), np.abs(predictor_scaled.coef_), label='Centrées-réduites')

num_features2 = X_scaled_dummies.shape[1]
plt.scatter(range(num_features2), np.abs(predictor_dummy.coef_), label='Avec one-hot', marker='v')
feature_names2 = df_dummies.drop(columns=['mpg', 'car name']).columns

plt.xlabel('Variable')
tmp = plt.xticks(range(num_features2), feature_names2, rotation=90)
tmp = plt.ylabel('Coefficient')

plt.legend(loc=(0.02, 0.75))